# Downloading Dataset from The National Gallery of Art
The gallery is sharing an art collection dataset with CC0, what we can use to train our CC0 ResNet backbone for other projects!

Their github page: https://github.com/NationalGalleryOfArt/opendata contains .csv files for the artworks, and we'll be downloading two of them and processing, downloading images of some of the artworks.

General data is stored in objects.csv data, and links to the images in published_images.csv data, they're joined together by uuid.

### Constants and imports

In [ ]:
import requests
import os
import random
import json
from tqdm.notebook import tqdm
import pandas as pd

CMA_GITHUB_OBJECT_DATA_CSV = 'https://github.com/NationalGalleryOfArt/opendata/raw/main/data/objects.csv'
CMA_GITHUB_IMAGES_DATA_CSV = 'https://github.com/NationalGalleryOfArt/opendata/raw/main/data/published_images.csv'
METADATA_DIRECTORY = 'metadata'
DATASET_NAME = 'nga'
RAW_METADATA_DIRECTORY = os.path.join(METADATA_DIRECTORY, 'raw')
RAW_METADATA_DIRECTORY_FILENAME = os.path.join(RAW_METADATA_DIRECTORY, f'{DATASET_NAME}.csv')
RAW_IMAGE_DIRECTORY = os.path.join('dataset', 'raw')
DATASET_IMAGES_LIMIT = 30000

### Preparing directories

In [ ]:
if not os.path.exists(RAW_IMAGE_DIRECTORY):
    os.makedirs(RAW_IMAGE_DIRECTORY)

if not os.path.exists(RAW_METADATA_DIRECTORY):
    os.makedirs(RAW_METADATA_DIRECTORY)
    
if not os.path.exists(METADATA_DIRECTORY):
    os.makedirs(METADATA_DIRECTORY)

### Downloading the raw metadata file onto the disk

### Loading the raw metadata file using pandas

### Checking existing metadata

### Filtering what we have to download

### Downloading it and updating the existing metadata file